## Install/import packages

In [1]:
!pip install pandas
!pip install langchain

import pandas as pd

## Load the unlabeled data into a dataframe

In [2]:
leadership_df = pd.read_csv('Mentor.Leadership.By.Semester.Combined.csv')

leadership_df

,Response,Semester
0,"Although Owens leadership skills are good, he ...",Fall 2017
1,Andrew should utilize colleagues to generate i...,Fall 2017
2,Bailey worked one part-time internship and her...,Fall 2017
3,BECAUSE I DIDN'T ALLOW FOR ANY EXPOSURE FOR CH...,Fall 2017
4,Ben's greatest area for improvement is in the ...,Fall 2017
...,...,...
1311,Only choosing this because unfortunately she w...,Fall 2022
1312,Sara could benefit from recognizing her worth ...,Fall 2022
1313,There was not much room to use leadership skills.,Fall 2022
1314,We do not have many leadership opportunities i...,Fall 2022


## Load the labeled data

In [3]:
labeled_df = pd.read_csv('Labelbox Cleaned-2020-10-22.csv')
labeled_df.rename(columns={'Final Label': 'Label'}, inplace=True)

labeled_df

,Labeled Data,Label,Dataset Name
0,NaN,NONE,Student
1,\n,NONE,Mentor
2,And then Alex can weigh in his ideas.,NONE,Student
3,"As Ally's career progresses, I have no doubt ...",NONE,Mentor
4,Caleb did a lot of the work filming and editi...,NONE,Student
...,...,...,...
6154,when discussing new project ideas for the futu...,STC,Student
6155,will also additionally take more responsibilit...,NONE,Student
6156,worked collaboratively with the study abroad o...,NONE,Student
6157,worked solo\n,NONE,Mentor


## Remove "Dataset Name" column from dataframe

In [4]:
labeled_df.drop(columns='Dataset Name', inplace=True)

labeled_df

,Labeled Data,Label
0,NaN,NONE
1,\n,NONE
2,And then Alex can weigh in his ideas.,NONE
3,"As Ally's career progresses, I have no doubt ...",NONE
4,Caleb did a lot of the work filming and editi...,NONE
...,...,...
6154,when discussing new project ideas for the futu...,STC
6155,will also additionally take more responsibilit...,NONE
6156,worked collaboratively with the study abroad o...,NONE
6157,worked solo\n,NONE


## Parse labeled responses for 2 of each label into JSON format

In [5]:
import json

def parse_responses_into_json():
    # Sample 2 rows for each unique label in the 'Label' column
    sampled_df = labeled_df.groupby('Label').apply(lambda x: x.sample(2)).reset_index(drop=True)

    # Convert df to JSON string
    json_responses = sampled_df.to_json(orient='records')
    
    # Convert TC and STC into "Transactional", TF and STF into "Transformational",
    # and IDK/NONE into "None".
    
    # Load the JSON string into a Python list of dictionaries
    data = json.loads(json_responses)

    # Iterate through each dictionary and change the "Label" value if it is "IDK"
    for item in data:
        if item["Label"] == "IDK" or item["Label"] == "NONE":
            item["Label"] = "None"
        elif item["Label"] == "TC" or item["Label"] == "STC":
            item["Label"] = "Transactional"
        elif item["Label"] == "TF" or item["Label"] == "STF":
            item["Label"] = "Transformational"     

    # Convert the list back to a JSON string
    relabeled_json_responses = json.dumps(data)
    
    return relabeled_json_responses

res = parse_responses_into_json()
print(res)

[{"Labeled Data": "My mentor has always called out great work done by her coworkers.", "Label": "None"}, {"Labeled Data": "We had an overall intern project for streamlined communication to early admitted students and I used my research to allow other interns to brainstorm ideas and the most effective ways to market to early admitted students.", "Label": "None"}, {"Labeled Data": "I feel the meeting went out very successfully.", "Label": "None"}, {"Labeled Data": "I feel that my leadership level has grown since I have worked here\n", "Label": "None"}, {"Labeled Data": "I was in charge of the hackathon\n", "Label": "Transactional"}, {"Labeled Data": "As a part of the School Book Project, I was tasked with assisting the project head collect funds that would provide summer reading books to all of the elementary school students of a local Title 1 school.", "Label": "Transactional"}, {"Labeled Data": "My team captain had very good leadership skills who believed in the concept of 'lead by exa

## Generate prompt

In [6]:
def generate_label_prompt(text):
    # Filter for 2 of each label
    labeled_data_json = parse_responses_into_json()

    combined_json_str = '[' + ', '.join(labeled_data_json) + ']'

    labeled_data = "# Labeled Data\n" + labeled_data_json + "\n"

    instructions = """# Instructions\n\"Based on the text in the 'Text' section below that contains \
an employer's evaluation of their intern's leadership skills, determine whether the text indicates a \
more transactional or transformational leadership style for the intern based on the definitions provided \
below in the 'Definitions' section. Labeled JSON data is provided in the 'Labeled Data' section as a reference \
for accurate labels of previous text examples. Give the output in a JSON format with 2 key value pairs where \
the 'Explanation' key corresponds to your reasoning for your choice and the 'Label' key corresponds to your \
choice for whether the text shows 'Transactional' or 'Transformational' leadership.\"\n"""
    text = "# Text\n\"" + text + "\""
    definitions = """# Definitions\n\"'Transactional leadership: characterized by a focus on hierarchy, organization, \
and performance. Leaders who exhibit this style often work within established procedures and promote compliance \
through rewards and punishments.' 'Transformational leadership: Transformational leaders inspire their followers \
to exceed their own self-interests for the good of the group or organization. They often encourage innovation, \
creativity, and personal development. They stimulate and inspire followers via charisma to both achieve extraordinary \
outcomes and develop their own leadership capacity.'\"\n"""
    new_prompt = instructions + definitions + labeled_data + text
    return new_prompt

prompt = generate_label_prompt(leadership_df["Response"][3])
print(prompt)

# Instructions
"Based on the text in the 'Text' section below that contains an employer's evaluation of their intern's leadership skills, determine whether the text indicates a more transactional or transformational leadership style for the intern based on the definitions provided below in the 'Definitions' section. Labeled JSON data is provided in the 'Labeled Data' section as a reference for accurate labels of previous text examples. Give the output in a JSON format with 2 key value pairs where the 'Explanation' key corresponds to your reasoning for your choice and the 'Label' key corresponds to your choice for whether the text shows 'Transactional' or 'Transformational' leadership."
# Definitions
"'Transactional leadership: characterized by a focus on hierarchy, organization, and performance. Leaders who exhibit this style often work within established procedures and promote compliance through rewards and punishments.' 'Transformational leadership: Transformational leaders inspire t

## Load the model and generate output based on prompt

In [7]:
!pip install -q transformers accelerate

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

device = "cuda" # the device to load the model onto

username = os.environ.get('USER')
cache_loc = os.path.join('/','scratch',username,'hf_cache')

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", cache_dir=cache_loc)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", cache_dir=cache_loc)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def generate_output(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    
    return decoded
    
output = generate_output(prompt)
print(output[0][len(prompt) + 20:])

cleaned_output = output[0].split('{')[1:]
cleaned_output = '{'.join(cleaned_output)
cleaned_output = cleaned_output.split('}')[:-1]
cleaned_output = '}'.join(cleaned_output)
cleaned_output = '{' + cleaned_output + '}'
cleaned_output

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
"Explanation": "The text does not show a clear focus on charisma, innovation, creativity, or personal development as required for transformational leadership. It instead shows a focus on the situation and the potential benefits/drawbacks of allowing exposure for Chinmay. The language used also seems to be more transactional, emphasizing compliance and following established procedures. Therefore, the label is 'Transactional'.",
"Label": "Transactional"
}</s>


'{"Labeled Data": "Using the whole team as his resource would help Shahab grow more\\n", "Label": "None"}, {"Labeled Data": "For instance, we had to run a story about the director of parking services, and we had to debate whether or not to portray him in a good light (leaving out the outrage over parking on campus), or to be honest about the way that parking services needs to improve to meet the needs of students and faculty.", "Label": "None"}, {"Labeled Data": "Because of not having the engineering background, Hannah was somewhat hesitant to take the lead with our STEM programs.", "Label": "None"}, {"Labeled Data": "After coordinating with the embassy in Amman, Jordan and our own IT department, we were able to get a beta-type system online.", "Label": "None"}, {"Labeled Data": "I learned a lot from him in regards to project management and how to lead everyone towards an end goal. \\n", "Label": "Transactional"}, {"Labeled Data": "This semester I had the pleasure of training my replac

# Load in pickled dataframe

In [10]:
import pandas as pd
# Load DataFrame from pickle file
model_output_df = pd.read_pickle('model_output.pkl')
model_output_df

,Labeled Data,Label,Model Output
0,NaN,NONE,"{\n""Explanation"": ""The provided text does not ..."
1,\n,NONE,"{\n""Explanation"": ""Based on the provided text,..."
2,And then Alex can weigh in his ideas.,NONE,"{\n""Explanation"": ""The text indicates that the..."
3,"As Ally's career progresses, I have no doubt ...",NONE,"{\n""Explanation"": ""The text implies that Ally'..."
4,Caleb did a lot of the work filming and editi...,NONE,
...,...,...,...
6154,when discussing new project ideas for the futu...,STC,
6155,will also additionally take more responsibilit...,NONE,
6156,worked collaboratively with the study abroad o...,NONE,
6157,worked solo\n,NONE,


In [11]:
# Count rows with strings in "Model Output" column
count_strings = model_output_df['Model Output'].apply(lambda x: isinstance(x, str) and x != '').sum()

print("Number of rows with strings:", count_strings)

Number of rows with strings: 4


## Generate output for each evaluation in labeled data and add it to dataframe

In [13]:
def is_valid_json(json_string):
    try:
        # Try to parse the JSON string
        json.loads(json_string)
        return True
    except json.JSONDecodeError:
        return False
    except TypeError:
        return False

In [14]:
def create_output_df(labeled_df):
    output_df = labeled_df
    output_df['Model Output'] = ''
    invalid_json_counts = {}

    for index, row in output_df.head(25).iterrows():
        invalid_json_count = 0
        
        # Generate prompt for current evaluation
        prompt = generate_label_prompt(str(row["Labeled Data"]))
        
        valid_json_flag = False
        
        while valid_json_flag == False:
            invalid_json_count += 1

            # Generate summary
            output = generate_output(prompt)

            # Parse output
            cleaned_output = output[0].split('[/INST] ')[-1]
            cleaned_output = cleaned_output.split('{')[1:]
            cleaned_output = '{'.join(cleaned_output)
            cleaned_output = cleaned_output.split('}')[:-1]
            cleaned_output = '}'.join(cleaned_output)
            cleaned_output = '{' + cleaned_output + '}'
            
            if cleaned_output == '{}':
                cleaned_output = output[0]

            if is_valid_json(cleaned_output) == True:
                valid_json_flag = True
                
                if invalid_json_count != 0:
                    invalid_json_counts[invalid_json_count] = invalid_json_counts.get(invalid_json_count, 0) + 1

        print("JSON not valid! Retrying!")
        # Add output to dataframe
        output_df.at[index, 'Model Output'] = cleaned_output
        
        # Save results
        filename = 'model_output_valid.pkl'
        output_df.to_pickle(filename)
        
    return output_df, invalid_json_counts
    
output_df, invalid_json_counts = create_output_df(labeled_df)
output_df

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


JSON not valid! Retrying!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


JSON not valid! Retrying!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


JSON not valid! Retrying!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


JSON not valid! Retrying!
JSON not valid! Retrying!


,Labeled Data,Label,Model Output
0,NaN,NONE,"{\n""Explanation"": ""The given text is not enoug..."
1,\n,NONE,"{\n ""Explanation"": ""The text indicates a mo..."
2,And then Alex can weigh in his ideas.,NONE,"{\n""Explanation"": ""The text suggests a more tr..."
3,"As Ally's career progresses, I have no doubt ...",NONE,"{\n""Explanation"": ""The text indicates a transf..."
4,Caleb did a lot of the work filming and editi...,NONE,"{""Label"": ""Transactional"", ""Explanation"": ""The..."
...,...,...,...
6154,when discussing new project ideas for the futu...,STC,
6155,will also additionally take more responsibilit...,NONE,
6156,worked collaboratively with the study abroad o...,NONE,
6157,worked solo\n,NONE,


In [16]:
import pickle

# Open a file in write-binary mode
with open('invalid_json_counts.pkl', 'wb') as file:
    # Pickle the dictionary and write it to the file
    pickle.dump(invalid_json_counts, file)

## Add model label column to output_df

In [18]:
output_df['IsValidJSON'] = output_df['Model Output'].apply(is_valid_json)

In [19]:
output_df.IsValidJSON.value_counts()

IsValidJSON
False    6134
True       25
Name: count, dtype: int64

# Get measure of model label accuracy compared to labeled data

## Parse model output for each eval in df for use in comparison

In [20]:
# Function to parse JSON and extract "Label"
def extract_label(model_output):
    # Isolate the JSON part of the text
    json_str_start = model_output.find('{')
    json_str_end = model_output.rfind('}') + 1
    json_str = model_output[json_str_start:json_str_end]

    try:
        json_data = json.loads(json_str)
        return json_data.get('Label', 'None')  # Return None if 'Label' key doesn't exist
    except json.JSONDecodeError:
        return 'None'  # Return None if JSON is invalid

# Apply the function to each row in 'Model Output'
output_df['Model Label'] = output_df['Model Output'].apply(extract_label)

output_df

,Labeled Data,Label,Model Output,IsValidJSON,Model Label
0,NaN,NONE,"{\n""Explanation"": ""The given text is not enoug...",True,None
1,\n,NONE,"{\n ""Explanation"": ""The text indicates a mo...",True,Transformational
2,And then Alex can weigh in his ideas.,NONE,"{\n""Explanation"": ""The text suggests a more tr...",True,Transformational
3,"As Ally's career progresses, I have no doubt ...",NONE,"{\n""Explanation"": ""The text indicates a transf...",True,Transformational
4,Caleb did a lot of the work filming and editi...,NONE,"{""Label"": ""Transactional"", ""Explanation"": ""The...",True,Transactional
...,...,...,...,...,...
6154,when discussing new project ideas for the futu...,STC,,False,None
6155,will also additionally take more responsibilit...,NONE,,False,None
6156,worked collaboratively with the study abroad o...,NONE,,False,None
6157,worked solo\n,NONE,,False,None


## Convert labeled_data labels to be comparable to model output labels

In [21]:
replaced_labels_df = pd.DataFrame(output_df)

# Define the mapping for label replacements
label_replacements = {
    'TC': 'Transactional',
    'STC': 'Transactional',
    'TF': 'Transformational',
    'STF': 'Transformational',
    'IDK': 'None',
    'NONE': 'None'
}

# Replace the values in the 'Label' column
replaced_labels_df['Label'] = replaced_labels_df['Label'].replace(label_replacements)

# Display the modified DataFrame
replaced_labels_df

,Labeled Data,Label,Model Output,IsValidJSON,Model Label
0,NaN,None,"{\n""Explanation"": ""The given text is not enoug...",True,None
1,\n,None,"{\n ""Explanation"": ""The text indicates a mo...",True,Transformational
2,And then Alex can weigh in his ideas.,None,"{\n""Explanation"": ""The text suggests a more tr...",True,Transformational
3,"As Ally's career progresses, I have no doubt ...",None,"{\n""Explanation"": ""The text indicates a transf...",True,Transformational
4,Caleb did a lot of the work filming and editi...,None,"{""Label"": ""Transactional"", ""Explanation"": ""The...",True,Transactional
...,...,...,...,...,...
6154,when discussing new project ideas for the futu...,Transactional,,False,None
6155,will also additionally take more responsibilit...,None,,False,None
6156,worked collaboratively with the study abroad o...,None,,False,None
6157,worked solo\n,None,,False,None


## Compare model output to labels in labeled_df (will need to convert labels to match)

In [22]:
# The number of rows to consider in the comparison
n = len(replaced_labels_df)
n = min(n, len(replaced_labels_df))

# Compare the columns and count matches for the first n rows
matches = (replaced_labels_df['Model Label'].iloc[:n] == replaced_labels_df['Label'].iloc[:n]).sum()

print(f"Number of matching labels in the first {n} rows: {matches}")

Number of matching labels in the first 6159 rows: 4107


In [23]:
replaced_labels_df.head(27)

,Labeled Data,Label,Model Output,IsValidJSON,Model Label
0,NaN,None,"{\n""Explanation"": ""The given text is not enoug...",True,None
1,\n,None,"{\n ""Explanation"": ""The text indicates a mo...",True,Transformational
2,And then Alex can weigh in his ideas.,None,"{\n""Explanation"": ""The text suggests a more tr...",True,Transformational
3,"As Ally's career progresses, I have no doubt ...",None,"{\n""Explanation"": ""The text indicates a transf...",True,Transformational
4,Caleb did a lot of the work filming and editi...,None,"{""Label"": ""Transactional"", ""Explanation"": ""The...",True,Transactional
5,I had to quickly deviate from my script and e...,None,"{\n""Explanation"": ""Transactional leadership is...",True,Transactional
6,Rather than holding a meeting about a problem...,None,"{\n""Explanation"": ""The text provided indicates...",True,Transactional
7,( The information is kinda vague because I don...,None,"{\n""Explanation"": ""Based on the given text, it...",True,Unknown
8,(Decipher and I share an office space.)\n,None,"{""Explanation"": ""The provided text does not co...",True,None
9,(We had issues with programmers missing deadli...,Transactional,"{\n""Explanation"": ""The text indicates a more t...",True,Transformational


In [24]:
# Save results
filename = 'replaced_labels_df1.pkl'
replaced_labels_df.to_pickle(filename)